In [19]:
import cv2
import os
import numpy as np # to pre-process data
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import time
import threading
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tqdm import tqdm
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array


In [20]:
class TqdmProgressCallback(Callback):

    def __init__(self, progress, status_label, update_ui_func):
        self.progress = progress
        self.status_label = status_label
        self.update_ui_func = update_ui_func

    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.progress["maximum"] = self.epochs
        self.progress["value"] = 0
        self.status_label.config(text="Status: Training in progress")
        self.update_ui_func()

    def on_epoch_end(self, epoch, logs=None):
        self.progress["value"] += 1
        self.update_ui_func()

    def on_train_end(self, logs=None):
        self.status_label.config(text="Status: Training completed")
        self.update_ui_func()


In [21]:
class FaceRecognitionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title("Face Recognition App")
        self.geometry("800x600")
        self.configure(bg="black")
        
        # store the recognition state
        self.recognition_active = False
        
        # Create a Frame widget to hold section content
        self.section_content = tk.Frame(self)
        self.section_content.pack(fill=tk.BOTH, expand=True)
        # Create a StringVar to store the selected section
        self.selected_section = tk.StringVar()
        self.selected_section.set("Data Collection")
        self.current_section = "Data Collection"
        # Create section buttons
        self.section_buttons = []
        self.section_widgets = {}

        self.sections = ["Data Collection", "Model Training", "Face Recognition","Information", "Privacy"]

        for section in self.sections:
            button = tk.Radiobutton(self, text=section, value=section, 
                                    variable=self.selected_section, command=self.change_section)
            button.pack(side=tk.LEFT, padx=10, pady=10)
            self.section_buttons.append(button)

        # Initialize the camera
        self.cap = cv2.VideoCapture(0)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        # Create the Data Collection section
        self.create_data_collection_section()

        # Initialize the first section
        self.change_section()
        
        self.dataset_path = tk.StringVar()
        
        # Define img_width and img_height attributes
        self.img_width = 224 # Replace with the width your model expects
        self.img_height = 224 # Replace with the height your model expects
        
        # Bind the on_closing method to the window close event
        self.protocol("WM_DELETE_WINDOW", self.on_closing)
        
        self.faces_captured = 0
        
    def create_data_collection_section(self):
        # Data Collection section
        self.data_collection_frame = ttk.Frame(self.section_content)
        self.section_widgets["Data Collection"] = self.data_collection_frame


        self.user_id_entry_label = tk.Label(self.data_collection_frame, text="User ID:")
        self.user_id_entry_label.pack(pady=5)

        self.user_id_entry = tk.Entry(self.data_collection_frame)
        self.user_id_entry.pack(pady=5)

        self.capture_button = tk.Button(self.data_collection_frame, text="Start Capturing Faces", command=self.start_capturing_faces)
        self.capture_button.pack(pady=10)

        self.preview_label = tk.Label(self.data_collection_frame, text="Preview")
        self.preview_label.pack(pady=10)

        self.preview_canvas = tk.Canvas(self.data_collection_frame, width=640, height=480, bg="black")
        self.preview_canvas.pack()
        
    def update_camera_feed(self):
        if not hasattr(self, "cap") or not self.cap.isOpened():
            self.cap = cv2.VideoCapture(0)
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        ret, frame = self.cap.read()
        if not ret:
            messagebox.showerror("Error", "Failed to read camera feed")
            return

        self.current_frame = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            # Get the face ROI
            roi = self.current_frame[y:y+h, x:x+w]
        if len(faces) > 0 and self.selected_section.get() == "Face Recognition" and self.recognition_active:
            predictions = self.recognize_faces(roi)
            label, confidence = predictions
            label_text = f"{label}: {confidence:.2f}%"
            cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame = ImageTk.PhotoImage(frame)
        if self.selected_section.get() in ["Data Collection"]:
            self.preview_canvas.create_image(0, 0, anchor=tk.NW, image=frame)
            self.preview_canvas.image = frame
        elif self.selected_section.get() in ["Face Recognition"]:
            self.face_recognition_canvas.create_image(0, 0, anchor=tk.NW, image=frame)
            self.face_recognition_canvas.image = frame

        self.after(30, self.update_camera_feed)

        
    def stop_camera_feed(self):
        if hasattr(self, "cap") and self.cap.isOpened():
            self.cap.release()
    
    def start_capturing_faces(self):
        user_id = self.user_id_entry.get().strip()
        if not user_id:
            messagebox.showerror("Error", "Please enter a User ID.")
            return

        self.user_id = user_id
        self.faces_captured = 0
        self.capture_face()
        
    def capture_face(self):
        if self.faces_captured < 20:
            gray = cv2.cvtColor(self.current_frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

            if len(faces) == 1:
                x, y, w, h = faces[0]
                face_roi = gray[y:y+h, x:x+w]
                self.save_face(face_roi, self.user_id)
                self.faces_captured += 1
                self.after(500, self.capture_face)
            else:
                self.after(30, self.capture_face)
        else:
            messagebox.showinfo("Success", f"Captured 20 face images for User ID: {self.user_id}")

    def save_face(self, face, user_id):
        faces_folder = os.path.join("faces", user_id)
        os.makedirs(faces_folder, exist_ok=True)
        face_count = len(os.listdir(faces_folder))
        face_path = os.path.join(faces_folder, f"face_{face_count}.jpg")
        cv2.imwrite(face_path, face)

    def create_model_training_section(self):
        self.model_training_frame = ttk.Frame(self.section_content)
        self.section_widgets["Model Training"] = self.model_training_frame
        
        self.dataset_label = tk.Label(self.model_training_frame, text="No dataset selected")
        self.dataset_label.pack(pady=5)
        
        self.select_dataset_button = tk.Button(self.model_training_frame, text="Select Dataset", command=self.select_dataset)
        self.select_dataset_button.pack(pady=5)
        
        self.model_name_label = tk.Label(self.model_training_frame, text="Model Name:")
        self.model_name_label.pack(pady=5)

        self.model_name_entry = tk.Entry(self.model_training_frame)
        self.model_name_entry.pack(pady=5)

        self.model_type_label = tk.Label(self.model_training_frame, text="Model Type:")
        self.model_type_label.pack(pady=5)

        self.selected_model = tk.StringVar()
        self.selected_model.set("CNN")  # set the default value
        self.model_type_menu = tk.OptionMenu(self.model_training_frame, self.selected_model, "CNN", "Eigenfaces", "LBPH")
        self.model_type_menu.pack(pady=5)

        self.train_button = tk.Button(self.model_training_frame, text="Start Model Training", command=self.start_model_training)
        self.train_button.pack(pady=10)

        self.progress = ttk.Progressbar(self.model_training_frame, orient=tk.HORIZONTAL, length=400, mode="determinate")
        self.progress.pack(pady=10)

        self.status_label = tk.Label(self.model_training_frame, text="Status: Not started", wraplength=400)
        self.status_label.pack(pady=5)
        
    def select_dataset(self):
        self.dataset_path.set(filedialog.askdirectory(initialdir=os.getcwd(), title="Select Dataset"))
        self.dataset_label.config(text=f"Selected dataset: {self.dataset_path.get()}")

            
    def preprocess_dataset_cnn(self, dataset_path):
        preprocessed_images = []
        labels = []
        label_dict = {}
        label_count = 0

        for subdir, dirs, files in os.walk(dataset_path):
            if len(files) > 0:
                if os.path.basename(subdir) not in label_dict:
                    label_dict[os.path.basename(subdir)] = label_count
                    label_count += 1

                for file in files:
                    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        # Preprocess the image
                        img = cv2.imread(os.path.join(subdir, file))
                        img = cv2.resize(img, (224, 224))
                        img = img.astype('float32') / 255

                        # Add the preprocessed image and its label to the lists
                        preprocessed_images.append(img)
                        labels.append(label_dict[os.path.basename(subdir)])

        preprocessed_images = np.array(preprocessed_images)
        labels = np.array(labels)

        return preprocessed_images, labels

    
    def train_cnn_model(self, X_train, X_test, y_train, y_test):
        # Convert labels to one-hot encoding
        num_classes = len(np.unique(y_train))
        y_train = tf.keras.utils.to_categorical(y_train, num_classes)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes)
        
        
        # Define the CNN model architecture
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(num_classes, activation='softmax')

        ])
        
        # Compile the model
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])
        
        # Train the model
        progress_callback = TqdmProgressCallback(
            self.progress, self.status_label, self.update_ui_during_training
        )
        
        history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[progress_callback], verbose=0)

        # Save the trained model
        model_name = self.model_name_entry.get().strip()
        if not model_name:
            model_name = 'cnn_model'
        model.save(f"{model_name}.h5")

        # Display training results
        messagebox.showinfo("Success", f"Model '{model_name}' trained and saved successfully.")

    def preprocess_dataset_eigenfaces(self, dataset_path):
        image_size = 96
        preprocessed_images = []
        labels = []
        for user_id in os.listdir(dataset_path):
            user_folder = os.path.join(dataset_path, user_id)
            for image_name in os.listdir(user_folder):
                image_path = os.path.join(user_folder, image_name)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (image_size, image_size))
                preprocessed_images.append(image)
                labels.append(user_id)
        return np.array(preprocessed_images), np.array(labels)

    def preprocess_dataset_lbph(self, dataset_path):
        preprocessed_images = []
        labels = []
        for user_id in os.listdir(dataset_path):
            user_folder = os.path.join(dataset_path, user_id)
            for image_name in os.listdir(user_folder):
                image_path = os.path.join(user_folder, image_name)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                preprocessed_images.append(image)
                labels.append(user_id)
        return preprocessed_images, np.array(labels)

    def train_and_save_model(self):
        dataset_path = self.dataset_path.get()
    
        if self.selected_model.get() == "CNN":
            preprocessed_images, labels = self.preprocess_dataset_cnn(dataset_path)
            # Add the implementation of the model training and saving for the CNN model here
            X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, labels, test_size=0.2, random_state=42, stratify=labels)
            self.train_cnn_model(X_train, X_test, y_train, y_test)
        elif self.selected_model.get() == "Eigenfaces":
            preprocessed_images, labels = self.preprocess_dataset_eigenfaces(dataset_path)
            # Add the implementation of the model training and saving for the Eigenfaces model here
        elif self.selected_model.get() == "LBPH":
            preprocessed_images, labels = self.preprocess_dataset_lbph(dataset_path)
            # Add the implementation of the model training and saving for the LBPH model here
        else:
            messagebox.showerror("Error", "Invalid model type selected.")
            
    def start_model_training(self):
        # 100ms delay for update in GUI for each epoch
        self.after(100, self.train_and_save_model)
    
    def update_ui_during_training(self):
        self.update_idletasks()
        self.update()
        
    def update_model_output(self, message):
        self.model_output.config(state=tk.NORMAL)
        self.model_output.insert(tk.END, message + "\n")
        self.model_output.config(state=tk.DISABLED)
        self.model_output.see(tk.END)
        

    def create_face_recognition_section(self):
        self.face_recognition_section = ttk.Frame(self.section_content)
        self.section_widgets["Face Recognition"] = self.face_recognition_section
        
        load_model_button = ttk.Button(self.face_recognition_section, text="Load Model", command=self.load_model_file)
        load_model_button.pack(padx=10, pady=10)

        self.loaded_model_label = tk.Label(self.face_recognition_section, text="No model loaded")
        self.loaded_model_label.pack(pady=5)
        
        self.start_face_recognition_button = ttk.Button(self.face_recognition_section, text="Recognise Face", command=self.start_face_recognition)
        self.start_face_recognition_button.pack(padx=10, pady=10)
        
        # Create a canvas to display the video feed
        self.face_recognition_canvas = tk.Canvas(self.face_recognition_section, width=640, height=480, bg="black")
        self.face_recognition_canvas.pack()
        #start_face_recognition(self)
        # Create a label to display the recognized person's name
        #self.recognized_person_label = tk.Label(self.face_recognition_section, text="", font=("Arial", 16))
        #self.recognized_person_label.pack(pady=10)


    def load_model_file(self):
        filetypes = [('Keras Model', '*.h5'), ('All files', '*')]
        model_path = filedialog.askopenfilename(initialdir=os.getcwd(), title="Select Model File", filetypes=filetypes)

        if not model_path:
            return

        try:
            self.model = load_model(model_path)
            self.loaded_model_label.config(text=f"Model loaded from {model_path}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load the model: {str(e)}")

    def preprocess_image(self, frame):
        # Preprocess the input frame for the CNN model
        frame = cv2.resize(frame, (self.img_width, self.img_height))
        frame = img_to_array(frame)
        frame = np.expand_dims(frame, axis=0)
        frame = frame.astype("float") / 255.0
        return frame

    def recognize_faces(self, frame):
        if not hasattr(self, 'model'):
            return []
        
        processed_frame = self.preprocess_image(frame)
        predictions = self.model.predict(processed_frame)
        
        # Check if the predictions array is not empty
        if len(predictions) > 0:
            label, confidence = np.argmax(predictions), np.max(predictions) * 100
            return label, confidence
        else:
            return None
        return predictions


    def start_face_recognition(self):
        self.recognition_active = True

        
    def create_information_section(self):
    # Information section
        self.information_frame = ttk.Frame(self.section_content)
        self.section_widgets["Information"] = self.information_frame
        # Information section content
        self.information_text = ("""This project is for educational purposes only, as part of a thesis. 
    The goal is to develop a face recognition system using AI and OpenCV, along with various recognition methods. 
    The datasets used for training and testing the model are obtained from images of classmates and family members.""")
        self.information_label = tk.Label(self.information_frame, text=self.information_text, wraplength=760, justify=tk.LEFT)
        self.information_label.pack(padx=20, pady=20)

    def create_privacy_section(self):
    # Privacy widgets
        self.privacy_frame = ttk.Frame(self.section_content)
        self.section_widgets["Privacy"] = self.privacy_frame
        # Privacy section content
        self.privacy_text = ("""The images and videos used in this project will not be shared with anyone. 
    They are solely used for training and testing the face recognition model. 
    All data handling and processing adhere to UK privacy regulations. 
    The project is committed to ensuring the privacy and security of the participants' data.""")
        self.privacy_label = tk.Label(self.privacy_frame, text=self.privacy_text, wraplength=760, justify=tk.LEFT)
        self.privacy_label.pack(padx=20, pady=20)
    
    
    def change_section(self):
        selected_section = self.selected_section.get()

        #for widget in self.section_content.winfo_children():
            #widget.pack_forget()
         # Hide the current section widget
        if hasattr(self, "current_section"):
            self.section_widgets[self.current_section].pack_forget()
        
        if selected_section not in self.section_widgets:
            if selected_section == "Data Collection":
                self.create_data_collection_section()
            elif selected_section == "Model Training":
                self.create_model_training_section()
            elif selected_section == "Face Recognition":
                self.create_face_recognition_section()
            elif selected_section == "Information":
                self.create_information_section()
            elif selected_section == "Privacy":
                self.create_privacy_section()
      #"""          
        if selected_section == "Data Collection":
            #self.create_data_collection_section()
            self.update_camera_feed()
            self.recognition_active = False
        elif selected_section == "Model Training":
            #self.create_model_training_section()
            self.recognition_active = False
        elif selected_section == "Face Recognition":
            #self.create_face_recognition_section()
            self.update_camera_feed()
        elif selected_section == "Information":
            #self.create_information_section()
            self.recognition_active = False
        elif selected_section == "Privacy":
            #self.create_privacy_section()
            self.recognition_active = False
        else:
            self.recognition_active = False
    
        # Show the selected section widget
        self.section_widgets[selected_section].pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
 
        self.current_section = selected_section


    
    def on_closing(self):
        self.cap.release()
        self.destroy()
    
if __name__ == "__main__":
    app = FaceRecognitionApp()
    app.mainloop()


In [ ]:
print(app.current_section)
print(app.recognition_active)
print(app.faces)

In [ ]:
app.predictions

In [4]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present
TrainingImagePath="faces"


In [5]:
TrainingImagePath

'faces'

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

In [8]:
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

In [9]:
# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 364 images belonging to 22 classes.


In [10]:
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 364 images belonging to 22 classes.


{'aassdd': 0,
 'aliraza': 1,
 'asasd': 2,
 'asda1': 3,
 'face1': 4,
 'face10': 5,
 'face11': 6,
 'face12': 7,
 'face13': 8,
 'face14': 9,
 'face15': 10,
 'face16': 11,
 'face2': 12,
 'face3': 13,
 'face4': 14,
 'face5': 15,
 'face6': 16,
 'face7': 17,
 'face8': 18,
 'face9': 19,
 'irtaza': 20,
 'testfaces1': 21}

In [11]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

In [12]:
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

In [13]:
# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

In [14]:
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'aassdd', 1: 'aliraza', 2: 'asasd', 3: 'asda1', 4: 'face1', 5: 'face10', 6: 'face11', 7: 'face12', 8: 'face13', 9: 'face14', 10: 'face15', 11: 'face16', 12: 'face2', 13: 'face3', 14: 'face4', 15: 'face5', 16: 'face6', 17: 'face7', 18: 'face8', 19: 'face9', 20: 'irtaza', 21: 'testfaces1'}

 The Number of output neurons:  22


In [15]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
'''Initializing the Convolutional Neural Network'''
classifier= Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''# STEP--3 FLattening'''
classifier.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 

In [16]:
###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
classifier.fit(
                    training_set,
                    steps_per_epoch=8,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/10
8/8 [==============================] - 13s 211ms/step - loss: 76.8298 - accuracy: 0.0464 - val_loss: 3.3841 - val_accuracy: 0.0594
Epoch 2/10
8/8 [==============================] - 1s 158ms/step - loss: 3.1939 - accuracy: 0.0657 - val_loss: 2.5825 - val_accuracy: 0.2625
Epoch 3/10
8/8 [==============================] - 1s 159ms/step - loss: 2.6305 - accuracy: 0.2646 - val_loss: 2.3551 - val_accuracy: 0.2594
Epoch 4/10
8/8 [==============================] - 1s 164ms/step - loss: 2.2179 - accuracy: 0.3295 - val_loss: 1.6082 - val_accuracy: 0.5063
Epoch 5/10
8/8 [==============================] - 1s 154ms/step - loss: 1.5932 - accuracy: 0.5544 - val_loss: 1.0877 - val_accuracy: 0.6906
Epoch 6/10
8/8 [==============================] - 1s 148ms/step - loss: 1.1150 - accuracy: 0.6676 - val_loss: 1.2097 - val_accuracy: 0.6562
Epoch 7/10
8/8 [==============================] - 1s 168ms/step - loss: 1.7148 - accuracy: 0.5074 - val_loss: 1.2464 - val_accuracy: 0.6906
Epoch 8/10
8/8 [==

In [17]:
import numpy as np
from keras.preprocessing import image
 
ImagePath='Face_Images/Final_Testing_Images/face4/3face4.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  face4
